# This notebook
- Creates simple RAY application
- Embed Transformer to provide summarization service.
- Deploys and tests locally
- Deploys and tests on Kubernetes cluster

### Require libraries
- ray[serve]==2.30.0
- numpy==1.26.4
- transformers==4.41.2
- torch==2.3.1

### Require container
- Need to have Kubernetes cluster 

In [ ]:
import requests
from ray import serve
from starlette.requests import Request
from transformers import pipeline

In [ ]:
# the Actor
@serve.deployment
class SummariserActor:
    
    def __init__(self, model_name):
        self.model = pipeline("summarization", model=model_name, tokenizer=model_name)

    def summarize(self, text: str) -> str:
        return self.model(text, min_length=5, max_length=10)

    async def __call__(self, http_request: Request) -> str:
        text = await http_request.json()
        return self.summarize(text['text'])

In [ ]:
# Local deploy and test
sum_app = SummariserActor.bind('t5-small')
sum_handle = serve.run(sum_app)

print(requests.get("http://127.0.0.1:8000/", json={"text": "The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering."}).text)

serve.shutdown()